In [1]:
from __future__ import absolute_import
from __future__ import print_function
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.optimizers import SGD, Adadelta, Adagrad
from keras.utils import np_utils, generic_utils
#from sklearn.cross_validation import train_test_split
import pandas as pd
import numpy as np
#from sklearn.cross_validation import train_test_split


In [2]:
batch_size = 32
nb_classes = 10
nb_epoch = 3
data_augmentation = False

In [3]:
# shape of the image (SHAPE x SHAPE)
shapex, shapey = 28, 28
# number of convolutional filters to use at each layer
nb_filters = [32, 64]
# level of pooling to perform at each layer (POOL x POOL)
nb_pool = [2, 2]
# level of convolution to perform at each layer (CONV x CONV)
nb_conv = [3, 3]
# the MNIST images are greyscale
image_dimensions = 1

In [4]:
train = pd.read_csv('mnist_train .csv')

In [5]:
Xcol = train.columns[1:]
ycol = 'label'
features = train[Xcol].values
labels = train[ycol].values

In [6]:
#pip install scikit-learn

In [7]:
# the data, shuffled and split between tran and test sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.3)
print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

X_train shape: (29400, 784)
29400 train samples
12600 test samples


In [8]:
# turn 1D feature vector into 3D cube
def makeImage(vectors):
    images = []
    nsamples = len(vectors[:, 0])
    for ivector in range(nsamples):
        vector = vectors[ivector, :]
        npixels = len(vector)
        nx = int(np.sqrt(npixels))
        ny = int(np.sqrt(npixels))
        image = np.zeros((nx, ny))
        for i in range(nx):
            for j in range(ny):
                image[j, i] = vector[j * nx + i]
        images.append(image)
    images = np.array(images)
    images = np.reshape(images, (nsamples, 1, nx, ny))
    return images

In [9]:
X_train_image = makeImage(X_train)

In [10]:
X_test_image = makeImage(X_test)

In [11]:
X_train_image.shape

(29400, 1, 28, 28)

In [12]:
# convert class vectors to binary class matrices
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)

In [13]:
y_train.shape, Y_train.shape

((29400,), (29400, 10))

In [14]:
model = Sequential()

In [16]:
from keras.models import Sequential
from keras.layers import Conv2D, Activation, MaxPooling2D, Flatten, Dense, Dropout

model = Sequential()

nb_filters = [32, 64]
nb_conv = [3, 3]
nb_pool = [2, 2]
shapex, shapey = 28, 28  # replace with the actual dimensions of your input images
nb_classes = 10  # replace with the actual number of classes
input_shape = (28, 28, 1)

# Input Layer
model.add(Conv2D(nb_filters[0], kernel_size=(nb_conv[0], nb_conv[0]), padding='valid', input_shape=input_shape))
model.add(Activation('relu'))

# Hidden Layer 1
model.add(Conv2D(nb_filters[1], kernel_size=(nb_conv[1], nb_conv[1]), padding='valid'))
model.add(Activation('relu'))

# Hidden Layer 2
model.add(Conv2D(nb_filters[1], kernel_size=(nb_conv[1], nb_conv[1])))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(nb_pool[1], nb_pool[1])))
model.add(Dropout(0.25))

# Fully Connected Layers
# Fully Connected Layers
# Fully Connected Layers
model.add(Flatten())
model.add(Dense(units=512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(units=nb_classes, activation='softmax'))


# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Print model summary
model.summary()


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 26, 26, 32)        320       
                                                                 
 activation_3 (Activation)   (None, 26, 26, 32)        0         
                                                                 
 conv2d_4 (Conv2D)           (None, 24, 24, 64)        18496     
                                                                 
 activation_4 (Activation)   (None, 24, 24, 64)        0         
                                                                 
 conv2d_5 (Conv2D)           (None, 22, 22, 64)        36928     
                                                                 
 activation_5 (Activation)   (None, 22, 22, 64)        0         
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 11, 11, 64)      

In [ ]:
#model.add(Convolution2D(nb_filters[1], nb_filters[0], nb_conv[0], nb_conv[0], border_mode='full'))
#model.add(Activation('relu'))
#model.add(Convolution2D(nb_filters[1], nb_filters[1], nb_conv[1], nb_conv[1]))
#model.add(Activation('relu'))
#model.add(MaxPooling2D(poolsize=(nb_pool[1], nb_pool[1])))
#model.add(Dropout(0.25))
#model.add(Flatten())
#model.add(Dense(nb_filters[-1] * (shapex / nb_pool[0] / nb_pool[1]) * (shapey / nb_pool[0] / nb_pool[1]), 512))
#model.add(Activation('relu'))
#model.add(Dropout(0.5))
#model.add(Dense(512, nb_classes))
#model.add(Activation('softmax'))

In [ ]:
# the image dimensions are the original dimensions divided by any pooling
# each pixel has a number of filters, determined by the last Convolution2D layer


In [19]:
# Define SGD optimizer with momentum
sgd = SGD(learning_rate=0.01, decay=1e-6, momentum=0.9, nesterov=True)

# Compile the model using SGD optimizer
model.compile(loss='categorical_crossentropy', optimizer=sgd)


In [26]:
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import generic_utils
import tensorflow as tf

if not data_augmentation:
    print("Not using data augmentation or normalization")

    X_train_image = X_train_image.astype("float32") / 255
    X_test_image = X_test_image.astype("float32") / 255

    model.fit(X_train_image, Y_train, batch_size=batch_size, epochs=10, verbose=1)
    score = model.evaluate(X_test_image, Y_test, batch_size=batch_size, verbose=1)
    print('Test loss:', score[0])
    print('Test accuracy:', score[1])

else:
    print("Using real-time data augmentation")

    # Define data augmentation parameters
    datagen = ImageDataGenerator(
        featurewise_center=True,
        samplewise_center=False,
        featurewise_std_normalization=True,
        samplewise_std_normalization=False,
        zca_whitening=False,
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        horizontal_flip=True,
        vertical_flip=False
    )

    # Compute quantities required for featurewise normalization
    datagen.fit(X_train)

    # Transpose dimensions to match model's expected input shape
    X_train_image = tf.transpose(X_train_image, perm=[0, 2, 3, 1])
    X_test_image = tf.transpose(X_test_image, perm=[0, 2, 3, 1])

    for epoch in range(10):
        print('-'*40)
        print('Epoch', epoch)
        print('-'*40)
        print("Training...")
        
        # Batch train with real-time data augmentation
        progbar = generic_utils.Progbar(X_train.shape[0])
        
        for X_batch, Y_batch in datagen.flow(X_train_image, Y_train, batch_size=batch_size):
            loss = model.train_on_batch(X_batch, Y_batch)
            progbar.add(X_batch.shape[0], values=[("train loss", loss)])

        print("Testing...")
        
        # Test time!
        progbar = generic_utils.Progbar(X_test.shape[0])
        
        for X_batch, Y_batch in datagen.flow(X_test_image, Y_test, batch_size=batch_size):
            score = model.test_on_batch(X_batch, Y_batch)
            progbar.add(X_batch.shape[0], values=[("test loss", score)])


Not using data augmentation or normalization
Epoch 1/10


ValueError: in user code:

    File "C:\Users\Hi\anaconda3\lib\site-packages\keras\engine\training.py", line 1284, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\Hi\anaconda3\lib\site-packages\keras\engine\training.py", line 1268, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\Hi\anaconda3\lib\site-packages\keras\engine\training.py", line 1249, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\Hi\anaconda3\lib\site-packages\keras\engine\training.py", line 1050, in train_step
        y_pred = self(x, training=True)
    File "C:\Users\Hi\anaconda3\lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "C:\Users\Hi\anaconda3\lib\site-packages\keras\engine\input_spec.py", line 298, in assert_input_compatibility
        raise ValueError(

    ValueError: Input 0 of layer "sequenti

In [ ]:
test = pd.read_csv('mnist_test.csv')

In [ ]:
test_image = makeImage(test.values)

In [ ]:
ypredict = model.predict(test_image, batch_size=batch_size)

In [ ]:
dfpredict = pd.DataFrame(np.argmax(ypredict, axis=1))
dfpredict.columns = ['Label']
dfpredict['ImageId'] = np.arange(28000) + 1
dfpredict.to_csv('../Data/predict_CNNbenchmark.csv', index=False)

In [ ]:
np.argmax(ypredict, axis=1)